# Clinical Trials Landscape Analysis

**Objective:**  
This project builds an end-to-end ETL pipeline using the ClinicalTrials.gov Data API. 
Clinical trial data is extracted via API calls, transformed and normalized using Python 
(pandas), loaded into a SQL database, and analyzed to uncover trends in trial activity, 
sponsor behavior, and completion outcomes.

**Key Questions:**
- How many stroke trials are completed vs ongoing?
- Where are stroke trials geographically concentrated?
- What types of interventions dominate stroke research?
- How large are stroke trials (enrollment)?
- What outcomes are most commonly measured?


## Data Source

- **API:** ClinicalTrials.gov Data API (v2)
- **Base URL:** https://clinicaltrials.gov/api/v2
- **Endpoint Used:** /studies
- **Authentication:** None (public API)
- **Response Format:** JSON

The ClinicalTrials.gov database is maintained by the U.S. National Library of Medicine 
and contains information on privately and publicly funded clinical studies conducted 
around the world.


## Project Architecture

1. Extract clinical trial data from the ClinicalTrials.gov API
2. Parse nested JSON fields into Pandas DataFrame
3. Transform and standardize clinical trial data using pandas (deduplication, type enforcement, and normalization)
4. Load cleaned data into a relational SQL database
5. Perform SQL-based analysis to answer research questions


In [ ]:
#Import libraries 
import os 
import json
import requests 
import pandas as pd 
import numpy as np
import sqlite3
from datetime import datetime
import time

In [ ]:
#url for the API
url = "https://clinicaltrials.gov/api/v2/studies"

PAGE_SIZE = 1000           # records per API call
MAX_PAGES = 10                # limit for development/testing

QUERY_TERM = "stroke"               # condition of interest
STATUS_FILTER = " "   # study completed or not 

In [ ]:
#parameters for the API call 
params = {"query.term": QUERY_TERM,
    "pageSize": PAGE_SIZE,
    }

EXTRACT 

In [ ]:
#response and response code 
response = requests.get(url, params)
response.status_code

In [ ]:
data = response.json()
data

Extract Data for Studies Table 



In [ ]:
#create column names and rows for DataFrame 
rows_studies = []
column_names_studies = [
    "nct_id",
    "brief_title",
    "official_title",
    "acronym",
    "org_name",
    "org_class",
    "overall_status",
    "start_date",
    "primary_completion_date",
    "completion_date",
    "study_first_post_date",
    "last_update_post_date",
    "study_type",
    "allocation",
    "intervention_model",
    "primary_purpose",
    "masking",
    "enrollment_count",
    "has_dmc",
    "fda_regulated_drug",
    "fda_regulated_device"
]


#loop through each study in the studies list and extract the necessary column values 
for study in data['studies']:
    
    # index into the json creating module variables, ensuring {} fallbacks at each level
    protocol                = study.get('protocolSection', {})
    id_module               = protocol.get('identificationModule', {})
    status_module           = protocol.get('statusModule', {})
    design_module           = protocol.get('designModule', {})
    oversight_module        = protocol.get('oversightModule', {})
    
    #assign values, ensuring {} fallbacks at each level to handle missing values (NULL)
    nct_id                  = id_module.get('nctId')
    brief_title             = id_module.get('briefTitle')
    official_title          = id_module.get('officialTitle')
    acronym                 = id_module.get('acronym')
    org_name                = id_module.get('organization', {}).get('fullName')
    org_class               = id_module.get('organization', {}).get('class')
    overall_status          = status_module.get('overallStatus')
    start_date              = status_module.get('startDateStruct', {}).get('date')
    primary_completion_date = status_module.get('primaryCompletionDateStruct', {}).get('date')
    completion_date         = status_module.get('completionDateStruct', {}).get('date')
    study_first_post_date   = status_module.get('studyFirstPostDateStruct', {}).get('date')
    last_update_post_date   = status_module.get('lastUpdatePostDateStruct', {}).get('date')
    study_type              = design_module.get('studyType')
    allocation              = design_module.get('designInfo', {}).get('allocation')
    intervention_model      = design_module.get('designInfo', {}).get('interventionModel')
    primary_purpose         = design_module.get('designInfo', {}).get('primaryPurpose')
    masking                 = design_module.get('designInfo', {}).get('maskingInfo', {}).get('masking')
    enrollment_count        = design_module.get('enrollmentInfo', {}).get('count')
    has_dmc                 = oversight_module.get('oversightHasDmc')
    fda_regulated_drug      = oversight_module.get('isFdaRegulatedDrug')
    fda_regulated_device    = oversight_module.get('isFdaRegulatedDevice')
    
    #initialize a tuple representing the active row for each iteration 
    active_row = (nct_id, brief_title, official_title, acronym, org_name, org_class, overall_status, start_date, 
                  primary_completion_date, completion_date, study_first_post_date, last_update_post_date, study_type,
                  allocation, intervention_model, primary_purpose, masking, enrollment_count, has_dmc, fda_regulated_drug, 
                  fda_regulated_device)    
    #append the active row to the rows list 
    rows_studies.append(active_row)
    
    


In [ ]:
df_studies = pd.DataFrame(rows_studies, columns=column_names_studies)

In [ ]:
df_studies.info()

In [ ]:
df_studies.head()

Extract Data for Study Conditions Table 

In [ ]:
rows_study_conditions = []

column_names_study_conditions = ['nct_id',
                                 'condition'
    
]

for study in data['studies']:
    
    protocol                = study.get('protocolSection', {})
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    condition               = protocol.get('conditionsModule', {}).get('conditions', [])
    
    #iterate through the list of conditions and for each iteration append the condition with the corresponding nct_id
    for condition in condition:
        rows_study_conditions.append(
            {'nct_id' : nct_id,
             'condition' : condition}
        )
    
    

In [ ]:
df_study_conditions = pd.DataFrame(rows_study_conditions, columns=column_names_study_conditions)

In [ ]:
df_study_conditions.info()

In [ ]:
df_study_conditions.head()

Extract Data for Study Interventions Table 

In [ ]:
rows_study_interventions = []
column_names_study_interventions = ['nct_id',
                           'intervention_name',
                           'intervention_type',
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    interventions           = protocol.get('armsInterventionsModule', {}).get('interventions', [])
    
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each intervention in the interventions list and append their name & type 
    #with their corresponding nct_id to the rows list 
    for intervention in interventions:
        rows_study_interventions.append({
            'nct_id' : nct_id,
            'intervention_name' : intervention.get('name'),
            'intervention_type' : intervention.get('type')
        })
    
    

In [ ]:
df_study_interventions = pd.DataFrame(rows_study_interventions, columns = column_names_study_interventions)

In [ ]:
df_study_interventions.info()

In [ ]:
df_study_interventions.head()

Extract Data for Primary Outcomes Table

In [ ]:
rows_primary_outcomes = []

column_names_primary_outcomes = [
    'nct_id',
    'outcome_measure',
    'time_frame',
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    primary_outcomes        = protocol.get('outcomesModule', {}).get('primaryOutcomes', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each primary outcome and append the measure & time frame with the corresponding nct_id
    #to the rows list
    for outcome in primary_outcomes:
        rows_primary_outcomes.append({
            'nct_id' : nct_id,
            'outcome_measure' : outcome.get('measure'),
            'time_frame' : outcome.get('timeFrame')
        })

In [ ]:
df_primary_outcomes = pd.DataFrame(rows_primary_outcomes, columns=column_names_primary_outcomes)

In [ ]:
df_primary_outcomes.info()

In [ ]:
df_primary_outcomes.head()

Extract Data for Secondary Outcomes Table 

In [ ]:
rows_secondary_outcomes = []

column_names_secondary_outcomes = [
    'nct_id',
    'outcome_measure',
    'time_frame'
]


for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    secondary_outcomes      = protocol.get('outcomesModule', {}).get('secondaryOutcomes', [])
    
    #iterate through each secondary outcome and append the measure & time frame with the corresponding nct_id
    #to the rows list
    for outcome in secondary_outcomes:
        rows_secondary_outcomes.append({
            'nct_id' : nct_id,
            'outcome_measure' : outcome.get('measure'),
            'time_frame' : outcome.get('timeFrame')
        })
    

In [ ]:
df_secondary_outcomes = pd.DataFrame(rows_secondary_outcomes, columns=column_names_secondary_outcomes)

In [ ]:
df_secondary_outcomes.info()

In [ ]:
df_secondary_outcomes.head()

Extract Data for Eligibility Summary Table 

In [ ]:
rows_eligibility_summary = []

column_names_eligibility_summary = [
    'nct_id',
    'sex',
    'min_age',
    'max_age',
    'healthy_volunteers'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    eligibility_module      = protocol.get('eligibilityModule', {})
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    sex                     = eligibility_module.get('sex')
    min_age                 = eligibility_module.get('minimumAge')
    max_age                 = eligibility_module.get('maximumAge')
    healthy_volunteers      = eligibility_module.get('healthyVolunteers')
    
    active_row = (nct_id, sex, min_age, max_age, healthy_volunteers)
    
    rows_eligibility_summary.append(active_row)

In [ ]:
df_eligibility_summary = pd.DataFrame(rows_eligibility_summary, columns=column_names_eligibility_summary)

In [ ]:
df_eligibility_summary.info()

In [ ]:
df_eligibility_summary.head()

Extract Data for Study Locations Table 

In [ ]:
rows_study_locations = []

column_names_study_locations = [
    'nct_id',
    'facility',
    'city',
    'country',
    'latitude',
    'longitude'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    locations_module        = protocol.get('contactsLocationsModule', {})
    locations               = locations_module.get('locations', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each location and append the facility, city, country, latitude, and longitude with the 
    #corresponding nct_id to the rows list 
    for location in locations:
        facility            = location.get('facility')
        city                = location.get('city')
        country             = location.get('country')
        latitude            = location.get('geoPoint', {}).get('lat')
        longitude           = location.get('geoPoint', {}).get('lon')
        
        active_row = (nct_id, facility, city, country, latitude, longitude)
        
        rows_study_locations.append(active_row)

In [ ]:
df_study_locations = pd.DataFrame(rows_study_locations, columns=column_names_study_locations)

In [ ]:
df_study_locations.info()

In [ ]:
df_study_locations.head()

Extract Data for Study References Table 

In [ ]:
rows_study_references = []

column_names_study_references = [
    'nct_id',
    'pmid',
    'citation'
]

for study in data['studies']:
    protocol                = study.get('protocolSection', {})
    references_module       = protocol.get('referencesModule', {})
    references              = references_module.get('references', [])
    
    nct_id                  = protocol.get('identificationModule', {}).get('nctId')
    
    #iterate through each references and append the pmid and citation with the corresponding nct_id to the rows list 
    for reference in references:
        pmid                = reference.get('pmid')
        citation            = reference.get('citation')
        
        active_row = (nct_id, pmid, citation)
        
        rows_study_references.append(active_row)

In [ ]:
df_study_references = pd.DataFrame(rows_study_references, columns=column_names_study_references)

In [ ]:
df_study_references.info()

In [ ]:
df_study_references.head()

Transform

Clean Studies Table 

Data Type Enforcement 

In [ ]:
#enforce date date type for date columns, coercing errors to NaT
date_cols = [
    'start_date',
    'primary_completion_date',
    'completion_date',
    'study_first_post_date',
    'last_update_post_date'
]

for col in date_cols:
    df_studies[col] = pd.to_datetime(df_studies[col], errors='coerce')

In [ ]:
#enforce boolean type for boolean columns
bool_cols = [
    'has_dmc',
    'fda_regulated_drug',
    'fda_regulated_device'  
]

for col in bool_cols:
    df_studies[col] = df_studies[col].astype(bool)

In [ ]:
df_studies['enrollment_count'] = pd.to_numeric(df_studies['enrollment_count'], errors='coerce')

Deduplication & Keys

In [ ]:
df_studies['nct_id'].is_unique

Business Logic Transformations

In [ ]:
#add a new column for the duration of the study in days 
df_studies['duration_days'] = (df_studies['completion_date'] - df_studies['start_date']).dt.days

In [ ]:
#add a new column for the enrollment size of the study in categories of (Small, Medium, Large, Very Large)
df_studies["enrollment_size"] = pd.cut(
    df_studies["enrollment_count"],
    bins=[0, 50, 200, 1000, float("inf")],
    labels=["Small", "Medium", "Large", "Very Large"]
)


In [ ]:
status_map = {
    "COMPLETED": "Completed",
    "RECRUITING": "Ongoing",
    "ACTIVE_NOT_RECRUITING": "Ongoing",
    "ENROLLING_BY_INVITATION": "Ongoing",
    "TERMINATED": "Stopped",
    "WITHDRAWN": "Stopped",
    "SUSPENDED": "Stopped"
}

df_studies["overall_status"] = df_studies["overall_status"].map(status_map)

Clean Study Conditions Table

Remove Bad Duplicates (Dimension Table)

In [ ]:
df_study_conditions = df_study_conditions.drop_duplicates(
    subset=["nct_id", "condition"]
)

Text Normalization

In [ ]:
df_study_conditions['condition'] = df_study_conditions['condition'].str.strip()

Clean Study Interventions Table 

Remove Bad Duplicates (Dimension Table)

In [ ]:
df_study_interventions = df_study_interventions.drop_duplicates(
    subset=['nct_id', 'intervention_name', 'intervention_type']
)

Text Normalization

In [ ]:
df_study_interventions["intervention_name"] = (
    df_study_interventions["intervention_name"]
    .str.strip()
)

df_study_interventions["intervention_type"] = (
    df_study_interventions["intervention_type"]
    .str.upper()
)

Clean Primary Outcomes Table 

Text Normalization

In [ ]:
df_primary_outcomes["outcome_measure"] = (
    df_primary_outcomes["outcome_measure"].str.strip()
)

Drop Bad Duplicates (Dimension Table)

In [ ]:
df_primary_outcomes = df_primary_outcomes.drop_duplicates(
    subset=["nct_id", "outcome_measure", "time_frame"]
)

Clean Secondary Outcomes Table 

Text Normalization (Dimension Table)

In [ ]:
df_secondary_outcomes['outcome_measure'] = (
    df_secondary_outcomes['outcome_measure'].str.strip()
)

Drop Bad Duplicates

In [ ]:
df_secondary_outcomes = df_secondary_outcomes.drop_duplicates(
    subset=['nct_id', 'outcome_measure', 'time_frame']
)

Clean Eligibility Summary Table

Standardize Age Fields

In [ ]:
#df_eligibility_summary['min_age'] = (df_eligibility_summary['min_age'].str.extract(r'(\d+)').astype(float))
#df_eligibility_summary['max_age'] = (df_eligibility_summary['max_age'].str.extract(r'(\d+)').astype(float))


Enforce Data Type (Boolean Type)

In [ ]:
df_eligibility_summary['healthy_volunteers'] = df_eligibility_summary['healthy_volunteers'].astype('boolean')
df_eligibility_summary.info()

Clean Study Locations Table 

Text Normalization 

In [ ]:
df_study_locations['facility'] = df_study_locations['facility'].str.strip()

df_study_locations['city'] = df_study_locations['city'].str.strip()

df_study_locations['country'] = df_study_locations['country'].str.strip()

df_study_locations

Remove Bad Duplicates 

In [ ]:
df_study_locations = df_study_locations.drop_duplicates(
    subset=["nct_id", "facility", "city", "country"]
)

Lat & Long numeric type enforcement

In [ ]:
df_study_locations.loc[:,'latitude'] = pd.to_numeric(df_study_locations['latitude'], errors='coerce') 

df_study_locations.loc[:,'longitude'] = pd.to_numeric(df_study_locations['longitude'], errors='coerce') 

Clean Study References Table

Remove Bad Duplicates

In [ ]:
df_study_references = df_study_references.drop_duplicates(
    subset=["nct_id", "pmid"]
)



ALL TABLES CLEANED!!!

In [ ]:
df_studies

In [ ]:
df_study_conditions

In [ ]:
df_study_interventions

In [ ]:
df_primary_outcomes

In [ ]:
df_secondary_outcomes

In [ ]:
df_eligibility_summary

In [ ]:
df_study_locations

In [ ]:
df_study_references

Upload to SQLite Database 

In [ ]:
import sqlite3
import pandas as pd 

#create connection to the SQLite database 
connection = sqlite3.connect('clinical_trials.db')

cursor = connection.cursor()

#create tables 
create_studies_table = """CREATE TABLE IF NOT EXISTS studies (
    nct_id TEXT PRIMARY KEY,
    brief_title TEXT, 
    official_title TEXT, 
    acronym TEXT, 
    org_name TEXT, 
    org_class TEXT, 
    overall_status TEXT, 
    start_date DATE,    
    primary_completion_date DATE,
    completion_date DATE,
    study_first_post_date DATE,
    last_update_post_date DATE,
    study_type TEXT,
    allocation TEXT,
    intervention_model TEXT,
    primary_purpose TEXT,
    masking TEXT,
    enrollment_count INTEGER,
    has_dmc BOOLEAN,
    fda_regulated_drug BOOLEAN,
    fda_regulated_device BOOLEAN,
    duration_days INTEGER,
    enrollment_size TEXT
    )"""
    
cursor.execute(create_studies_table)

#create study_conditions table 
create_study_conditions_table = """CREATE TABLE IF NOT EXISTS study_conditions (
    nct_id TEXT,
    condition TEXT,
    PRIMARY KEY (nct_id, condition),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_conditions_table)

#create study_interventions table
create_study_interventions_table = """CREATE TABLE IF NOT EXISTS study_interventions (
    nct_id TEXT,
    intervention_name TEXT,
    intervention_type TEXT,
    PRIMARY KEY (nct_id, intervention_name, intervention_type),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_interventions_table)

#create primary_outcomes table
create_primary_outcomes_table = """CREATE TABLE IF NOT EXISTS primary_outcomes (
    nct_id TEXT,
    outcome_measure TEXT,
    time_frame TEXT,
    PRIMARY KEY (nct_id, outcome_measure, time_frame),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""    
    
cursor.execute(create_primary_outcomes_table)   

#create secondary_outcomes table
create_secondary_outcomes_table = """CREATE TABLE IF NOT EXISTS secondary_outcomes (
    nct_id TEXT,
    outcome_measure TEXT,
    time_frame TEXT,
    PRIMARY KEY (nct_id, outcome_measure, time_frame),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""

cursor.execute(create_secondary_outcomes_table)

#create eligibility_summary table
create_eligibility_summary_table = """CREATE TABLE IF NOT EXISTS eligibility_summary (
    nct_id TEXT PRIMARY KEY,
    sex TEXT,
    min_age TEXT,   
    max_age TEXT,
    healthy_volunteers BOOLEAN,
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""

cursor.execute(create_eligibility_summary_table)

#create study_locations table
create_study_locations_table = """CREATE TABLE IF NOT EXISTS study_locations (
    nct_id TEXT,
    facility TEXT,
    city TEXT,
    country TEXT,
    latitude REAL,
    longitude REAL,
    PRIMARY KEY (nct_id, facility, city, country),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_locations_table)

#create study_references table
create_study_references_table = """CREATE TABLE IF NOT EXISTS study_references (
    nct_id TEXT,
    pmid TEXT,
    citation TEXT,
    PRIMARY KEY (nct_id, pmid),
    FOREIGN KEY (nct_id) REFERENCES studies(nct_id)
    )"""
    
cursor.execute(create_study_references_table)

#commit the changes to the database
connection.commit()

In [ ]:
#load DataFrames into their respective tables in the database 

df_studies.to_sql('studies', connection, if_exists='replace', index=False)

df_study_conditions.to_sql('study_conditions', connection, if_exists='replace', index=False)

df_study_interventions.to_sql('study_interventions', connection, if_exists='replace', index=False)

df_primary_outcomes.to_sql('primary_outcomes', connection, if_exists='replace', index=False)

df_secondary_outcomes.to_sql('secondary_outcomes', connection, if_exists='replace', index=False)

df_eligibility_summary.to_sql('eligibility_summary', connection, if_exists='replace', index=False)

df_study_locations.to_sql('study_locations', connection, if_exists='replace', index=False)

df_study_references.to_sql('study_references', connection, if_exists='replace', index=False)


In [ ]:
# Export views as xlsx to connect to tableau

df_study_overview = pd.read_sql_query('SELECT * FROM study_overview', connection)
df_study_overview.to_excel('study_overview.xlsx', index=False)

df_intervention_breakdown = pd.read_sql_query('SELECT * FROM intervention_breakdown', connection)
df_intervention_breakdown.to_excel('intervention_breakdown.xlsx', index=False)

df_geography_breakdown = pd.read_sql_query('SELECT * FROM geography_breakdown', connection)
df_geography_breakdown.to_excel('geography_breakdown.xlsx', index=False)

df_condition_breakdown = pd.read_sql_query('SELECT * FROM condition_breakdown', connection)
df_condition_breakdown.to_excel('condition_breakdown.xlsx', index=False)

connection.close()